# XGBoost - 베이스라인 버전 (one-hot encoding)

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sample_submission.csv
/kaggle/input/train.csv
/kaggle/input/test.csv


In [4]:
data_path = '/kaggle/input/'

train = pd.read_csv(data_path + 'train.csv', index_col= 'ID')
test = pd.read_csv(data_path + 'test.csv', index_col= 'ID')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col= 'ID')

## 1. 데이터 통합

In [5]:
all_data = pd.concat([train, test], ignore_index= True)
all_data = all_data.drop('임신 성공 여부', axis= 1)

In [6]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [7]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    all_data[col] = all_data[col].astype(str)

## 2. 피처 엔지니어링: 원 핫 인코딩

In [8]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder()

encoded_cat_matrix = onehot_encoder.fit_transform(all_data[categorical_columns])

encoded_cat_matrix

<346418x205 sparse matrix of type '<class 'numpy.float64'>'
	with 16281646 stored elements in Compressed Sparse Row format>

In [9]:
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

In [10]:
all_columns = all_data.columns
all_columns

Index(['시술 시기 코드', '시술 당시 나이', '임신 시도 또는 마지막 임신 경과 연수', '시술 유형', '특정 시술 유형',
       '배란 자극 여부', '배란 유도 유형', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부',
       '착상 전 유전 진단 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인',
       '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환',
       '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', '불임 원인 - 여성 요인', '불임 원인 - 자궁경부 문제',
       '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인',
       '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '배아 생성 주요 이유', '총 시술 횟수',
       '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수',
       'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '총 생성 배아 수',
       '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수',
       '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수',
       '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '난자 출처', '정자 출처',
       '난자 기증자 나이', '정자 기증자 나이', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부',
       '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자

In [11]:
len(all_columns)

67

In [12]:
from scipy import sparse

extracted_data_sprs = sparse.hstack([sparse.csr_matrix(all_data[numeric_columns]),
                               encoded_cat_matrix],
                              format='csr')

## 3. 데이터 나누기

In [13]:
num_train = len(train) 

X = extracted_data_sprs[:num_train]
X_test = extracted_data_sprs[num_train:]

y = train['임신 성공 여부'].values

## 4. 파라미터 최적화

In [14]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

# 8:2 비율로 훈련 데이터, 검증 데이터 분리 (베이지안 최적화 수행용)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.2, 
                                                      random_state=0)
# 베이지안 최적화용 데이터셋
bayes_dtrain = xgb.DMatrix(X_train, y_train)
bayes_dvalid = xgb.DMatrix(X_valid, y_valid)

In [15]:
# 베이지안 최적화를 위한 하이퍼파라미터 범위
# TODO: 점점 좁혀나가자!!
param_bounds = {
    'max_depth': (6, 8),
    'subsample': (0.65, 0.9),
    'colsample_bytree': (0.8, 1.0),
    'min_child_weight': (6, 7),
    'gamma': (8, 10),
    'reg_alpha': (7, 9),
    'reg_lambda': (1.1, 1.5),
    'scale_pos_weight': (1.4, 1.6)
}

# 값이 고정된 하이퍼파라미터
fixed_params = {'objective': 'binary:logistic',
                'learning_rate': 0.02,
                'random_state': 1991}

In [16]:
from sklearn.metrics import roc_auc_score

# XGBoost 커스텀 평가 함수
def xgb_roc_auc(y_pred, dtrain):
    y_true = dtrain.get_label()
    return "roc_auc", roc_auc_score(y_true, y_pred)

In [17]:
from sklearn.metrics import roc_auc_score
from lightgbm import early_stopping

def eval_function(max_depth, subsample, colsample_bytree, min_child_weight,
                 reg_alpha, gamma, reg_lambda, scale_pos_weight):
    '''최적화하려는 평가지표(roc_auc) 계산 함수'''
    # 베이지안 최적화를 수행할 하이퍼파라미터
    params = {'max_depth': int(round(max_depth)),
              'subsample': subsample,
              'colsample_bytree': colsample_bytree,
              'min_child_weight': min_child_weight,
              'gamma': gamma,
              'reg_alpha':reg_alpha,
              'reg_lambda': reg_lambda,
              'scale_pos_weight': scale_pos_weight}
    # 값이 고정된 하이퍼파라미터도 추가
    params.update(fixed_params)
    
    print('하이퍼파라미터 :', params)    
        
    # XGBoost 모델 훈련
    xgb_model = xgb.train(params=params, 
                          dtrain=bayes_dtrain,
                          num_boost_round=2000,
                          evals=[(bayes_dvalid, 'bayes_dvalid')],
                          maximize=True,
                          custom_metric=xgb_roc_auc, 
                          callbacks=[xgb.callback.EarlyStopping(rounds=200)],
                          verbose_eval=0)
                           
    best_iter = xgb_model.best_iteration # 최적 반복 횟수
    # 검증 데이터로 예측 수행
    preds = xgb_model.predict(bayes_dvalid, 
                              iteration_range=(0, best_iter))
    # roc_auc 계산
    roc_auc = roc_auc_score(bayes_dvalid.get_label(), preds)
    print(f'roc_auc : {roc_auc}\n')
    
    return roc_auc

In [18]:
from bayes_opt import BayesianOptimization

# 베이지안 최적화 객체 생성
optimizer = BayesianOptimization(f=eval_function, 
                                 pbounds=param_bounds, 
                                 random_state=0)

In [19]:
# 베이지안 최적화 수행
# TODO: init_points 10~15, n_iter 30~70
optimizer.maximize(init_points=15, n_iter=60)

|   iter    |  target   | colsam... |   gamma   | max_depth | min_ch... | reg_alpha | reg_la... | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------
하이퍼파라미터 : {'max_depth': 7, 'subsample': 0.87294325019552, 'colsample_bytree': 0.909762700785465, 'min_child_weight': 6.544883182996897, 'gamma': 9.430378732744838, 'reg_alpha': 7.84730959867781, 'reg_lambda': 1.3583576452266626, 'scale_pos_weight': 1.4875174422525386, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}
roc_auc : 0.7394381654769072

| 1         | 0.7394    | 0.9098    | 9.43      | 7.206     | 6.545     | 7.847     | 1.358     | 1.488     | 0.8729    |
하이퍼파라미터 : {'max_depth': 8, 'subsample': 0.6717823249253851, 'colsample_bytree': 0.9927325521002058, 'min_child_weight': 6.528894919752904, 'gamma': 8.766883037651555, 'reg_alpha': 8.136089122187865, 'reg_lambda': 1.4702386553170643, 'scale_pos_weight': 1.414

In [20]:
# 평가함수 점수가 최대일 때 하이퍼파라미터
max_params = optimizer.max['params']
max_params

{'colsample_bytree': 0.83100305182246,
 'gamma': 8.0503356372611,
 'max_depth': 7.714205395057098,
 'min_child_weight': 6.617094996274681,
 'reg_alpha': 7.3168709375076535,
 'reg_lambda': 1.2463377314671225,
 'scale_pos_weight': 1.5214260635820143,
 'subsample': 0.879872640465336}

In [21]:
# 정수형 하이퍼파라미터 변환
max_params['max_depth'] = int(round(max_params['max_depth']))

# 값이 고정된 하이퍼파라미터 추가
max_params.update(fixed_params)

In [22]:
max_params

{'colsample_bytree': 0.83100305182246,
 'gamma': 8.0503356372611,
 'max_depth': 8,
 'min_child_weight': 6.617094996274681,
 'reg_alpha': 7.3168709375076535,
 'reg_lambda': 1.2463377314671225,
 'scale_pos_weight': 1.5214260635820143,
 'subsample': 0.879872640465336,
 'objective': 'binary:logistic',
 'learning_rate': 0.02,
 'random_state': 1991}

## 5. 모델 훈련

In [ ]:
from sklearn.model_selection import StratifiedKFold

# 층화 K 폴드 교차 검증기 생성
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=1991)

# OOF 방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_val_preds = np.zeros(X.shape[0]) 
# OOF 방식으로 훈련된 모델로 테스트 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_test_preds = np.zeros(X_test.shape[0]) 

# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    # 각 폴드를 구분하는 문구 출력
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
    # 훈련용 데이터, 검증용 데이터 설정
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]

    # XGBoost 전용 데이터셋 생성 
    dtrain = xgb.DMatrix(X_train, y_train)
    dvalid = xgb.DMatrix(X_valid, y_valid)
    dtest = xgb.DMatrix(X_test)
    # XGBoost 모델 훈련
    xgb_model = xgb.train(params=max_params, 
                          dtrain=dtrain,
                          num_boost_round=2000,
                          evals=[(dvalid, 'valid')],
                          maximize=True,
                          custom_metric=xgb_roc_auc, 
                          callbacks=[xgb.callback.EarlyStopping(rounds=200)],
                          verbose_eval=0)

    # 모델 성능이 가장 좋을 때의 부스팅 반복 횟수 저장
    best_iter = xgb_model.best_iteration
    # 테스트 데이터를 활용해 OOF 예측
    oof_test_preds += xgb_model.predict(dtest,
                                        iteration_range=(0, best_iter))/folds.n_splits
    
    # 모델 성능 평가를 위한 검증 데이터 타깃값 예측 
    oof_val_preds[valid_idx] += xgb_model.predict(dvalid, 
                                                  iteration_range=(0, best_iter))
    
    # 검증 데이터 예측 확률에 대한 ROC-AUC
    roc_auc = roc_auc_score(dvalid.get_label(), oof_val_preds[valid_idx])
    print(f'폴드 {idx+1} roc-auc : {roc_auc}\n')

## 6. 최종 결과

In [ ]:
print('OOF 검증 데이터 roc-auc :', roc_auc_score(y, oof_val_preds))

In [ ]:
submission['probability'] = oof_test_preds
submission.to_csv('xgb_onehot_submission.csv')